In [1]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import vtk
import os
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D

plt.style.use('default')

np.random.seed(23765)

In [2]:
column_headers_pos = ['local id', 'pos x', 'pos y', 'pos z', 'area', 'type']

df_pos = pd.read_csv('rank_0_positions.txt', skiprows= 8, delimiter=' ', names=column_headers_pos)
df_pos.head()


,local id,pos x,pos y,pos z,area,type
0,1,88.017654,143.88911,83.259131,area_43,ex
1,2,88.071787,143.79220,83.237149,area_43,ex
2,3,88.093434,143.95190,83.168331,area_43,ex
3,4,88.063898,143.81981,83.317563,area_43,ex
4,5,88.066131,143.97354,83.238470,area_43,ex


In [3]:
areas = df_pos['area'].unique()
print(areas)

['area_43' 'area_30' 'area_9' 'area_5' 'area_41' 'area_46' 'area_35'
 'area_34' 'area_45' 'area_8' 'area_14' 'area_4' 'area_1' 'area_32'
 'area_24' 'area_22' 'area_6' 'area_12' 'area_42' 'area_47' 'area_21'
 'area_25' 'area_38' 'area_20' 'area_26' 'area_39' 'area_0' 'area_16'
 'area_23' 'area_33' 'area_28' 'area_27' 'area_31' 'area_13' 'area_15'
 'area_2' 'area_18' 'area_11' 'area_19' 'area_29' 'area_44' 'area_40'
 'area_36' 'area_7' 'area_17' 'area_37' 'area_10' 'area_3']


In [4]:
neurons = []
#df_pos_reduced = pd.DataFrame()

for area in areas:
    df_area = df_pos[df_pos['area'] == area]
    local_neurons = df_area['local id'].unique()
    chosen_neurons = pd.DataFrame(np.random.choice(local_neurons, 5))

    neurons.append(chosen_neurons)
    #df_pos_reduced = pd.concat([df_pos_reduced, chosen_neurons], ignore_index=True)

#df_pos_reduced.head()

In [5]:
#neurons = np.sort(np.random.choice(np.arange(0, 50000, 1), size=100, replace=False))

In [6]:
column_headers_monitors = ['step', 'calcium']

df_calcium = pd.DataFrame()

for i in neurons:
    for j in i.to_numpy():
        filepath = f'/Users/Kevin/Documents/Opleiding/Master/Computational_Science/Year_2/Semester_1/Period_2/Scientific_Visualisation_and_Virtual_Reality/Project/SciVisContest23/SciVisContest23/viz-stimulus/monitors/0_{j[0]}.csv'
        temp_df = pd.read_csv(filepath, skiprows=0, usecols=[0, 5], delimiter=';', names=column_headers_monitors)
        temp_df['neuron_id'] = j*np.ones(len(temp_df['calcium']), dtype=int)  # Add the neuron ID as a new column
        df_calcium = pd.concat([df_calcium, temp_df], ignore_index=True)

df_calcium.tail()

,step,calcium,neuron_id
2399995,9500,0.689080,49623
2399996,9600,0.688189,49623
2399997,9700,0.687308,49623
2399998,9800,0.688420,49623
2399999,9900,0.689528,49623


In [7]:
Ca_area = pd.DataFrame([], ['Calcium_concentration'])
average_Ca = []

for ix, i in enumerate(neurons):
    average_Ca = np.mean(i[0].to_numpy())
    Ca_area[areas[ix]] = average_Ca

Ca_area.head()


,area_43,area_30,area_9,area_5,area_41,area_46,area_35,area_34,area_45,area_8,...,area_19,area_29,area_44,area_40,area_36,area_7,area_17,area_37,area_10,area_3
Calcium_concentration,1723.8,2034.4,1113.8,8882.0,4747.6,6529.6,11143.6,12457.2,12184.2,17500.0,...,46228.2,43096.6,42328.6,44816.6,42906.6,44639.4,46140.8,46752.4,48473.0,49219.8


In [ ]:
# Create a vtkPoints object to store neuron positions
points = vtk.vtkPoints()
for _, row in df_pos.iterrows():
    if row['local id'] in neurons:
        points.InsertNextPoint(row['pos x'], row['pos y'], row['pos z'])

# Create a vtkPolyData object to store the points
polydata = vtk.vtkPolyData()
polydata.SetPoints(points)

# Create a mapper and actor for the points
vertex_filter = vtk.vtkVertexGlyphFilter()  # Ensures points are rendered
vertex_filter.SetInputData(polydata)
vertex_filter.Update()

mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(vertex_filter.GetOutputPort())

actor = vtk.vtkActor()
actor.SetMapper(mapper)

# Customize point appearance
actor.GetProperty().SetColor(1.0, 1.0, 1.0)  # White points
actor.GetProperty().SetPointSize(2)         # Set point size

# Set up the renderer, render window, and interactor
renderer = vtk.vtkRenderer()
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)

interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)

# Add the actor to the renderer
renderer.AddActor(actor)
renderer.SetBackground(0.1, 0.1, 0.1)  # Background color: dark gray

# Start the visualization
render_window.Render()
interactor.Start()

# Neuron calcium concentrations plot using Matplotlib

In [ ]:
# Specify the time step to visualize
time_point = 100000
df_calcium_at_time = df_calcium[df_calcium['step'] == time_point]

# Merge position and calcium data
df_merged = pd.merge(df_pos, df_calcium_at_time, left_on='local id', right_on='neuron_id')

fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Normalize calcium concentration for coloring
global_min = df_calcium['calcium'].min()  # Use global min and max for consistent coloring
global_max = df_calcium['calcium'].max()
calcium_values = df_merged['calcium']
colors = plt.cm.viridis((calcium_values - global_min) / (global_max - global_min))

scatter = ax.scatter(
    df_merged['pos x'], df_merged['pos y'], df_merged['pos z'],
    c=colors, s=50, cmap='viridis', edgecolor='k'
)

cbar = fig.colorbar(scatter, ax=ax, shrink=0.5, aspect=10)
cbar.set_label('Calcium Concentration')

ax.set_title(f'Neuron Calcium Concentrations at Time Step {time_point}')

plt.show()

# Neuron calcium concentrations plot using VTK

In [ ]:
# Specify the time step to visualize
time_point = 10000  # Replace with your desired time step

# Filter data for the specified time point
df_calcium_at_time = df_calcium[df_calcium['step'] == time_point]

# Merge position and calcium data
df_merged = pd.merge(df_pos, df_calcium_at_time, left_on='local id', right_on='neuron_id')

# Normalize calcium concentration for coloring
global_min = df_calcium['calcium'].min()  # Use global range
global_max = df_calcium['calcium'].max()
calcium_values = df_merged['calcium']
normalized_calcium = (calcium_values - global_min) / (global_max - global_min)

# Create a VTK points object to hold neuron positions
points = vtk.vtkPoints()
for _, row in df_merged.iterrows():
    points.InsertNextPoint(row['pos x'], row['pos y'], row['pos z'])

# Create a VTK cell array to represent vertices
vertices = vtk.vtkCellArray()
for i in range(len(df_merged)):
    vertices.InsertNextCell(1)
    vertices.InsertCellPoint(i)

# Create a VTK polydata object to store the points and vertices
polydata = vtk.vtkPolyData()
polydata.SetPoints(points)
polydata.SetVerts(vertices)

# Add calcium concentration as a scalar field for coloring
calcium_array = vtk.vtkFloatArray()
calcium_array.SetName("Calcium")
for value in normalized_calcium:
    calcium_array.InsertNextValue(value)
polydata.GetPointData().SetScalars(calcium_array)

# Create a mapper and actor for visualization
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputData(polydata)
mapper.SetScalarRange(0.0, 1.0)  # Normalized range [0, 1]
mapper.SetColorModeToMapScalars()
mapper.SetScalarModeToUsePointData()
mapper.Update()

actor = vtk.vtkActor()
actor.SetMapper(mapper)

actor.GetProperty().SetPointSize(10)  # Adjust the size as needed

# Create a renderer, render window, and interactor
renderer = vtk.vtkRenderer()
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)

interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(render_window)

# Add the actor to the renderer
renderer.AddActor(actor)
renderer.SetBackground(0.1, 0.1, 0.1)  # Dark gray background

# Start the visualization
render_window.Render()
interactor.Start()